# villaInventorySdk
[docs](https://thanakijwanavit.github.io/villa-inventory-sdk/)
> read and write inventory in real time

## Install

`pip install villaInventorySdk`

## How to use

Uploading a large amount of data

In [ ]:
#hide
isAws = True
if isAws:
  USER = None
  PW = None
else:
  import pickle
  KEY = ''
  PW = ''
  keypath = '/Users/nic/.villa-product-master-inventory'
  if KEY and PW:
    with open (keypath, 'wb') as f:
      pickle.dump({
          'KEY': KEY,
          'PW': PW
      }, f)
  with open(keypath, 'rb') as f:
    creden = pickle.load(f)
  USER = creden['KEY']
  PW = creden['PW']

## sample input

In [ ]:
sampleInput = [ 
  {'ib_brcode': '1023', 'ib_cf_qty': '835', 'ib_prcode': '84621', 'new_ib_vs_stock_cv': '839'},
  {'ib_brcode': '1022', 'ib_cf_qty': '24', 'ib_prcode': '12424', 'new_ib_vs_stock_cv': '21'}
]

## Upload data

In [ ]:
#hide
from villaInventorySdk.inventory import InventorySdk
from random import randrange
import boto3, time, json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from datetime import datetime

In [ ]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'
branch = 'dev-manual'

In [ ]:
#hide
inputBucketName= 'input-bucket-test-inventory-6b3753e2f2'
functionName= 'update-inventory-s3-test-inventory-6b3753e2f2'
invocationType = 'RequestResponse'

In [ ]:
#hide
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r took %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

## init sdk

In [ ]:
%%time
sdk = InventorySdk(user=USER, pw=PW, branchName = branch)

CPU times: user 36.6 ms, sys: 4.73 ms, total: 41.3 ms
Wall time: 1.05 s


## Update inventory 

In [ ]:
%%time
sdk.updateWithS3( sampleInput )

CPU times: user 53.4 ms, sys: 2.56 ms, total: 55.9 ms
Wall time: 322 ms


{'body': 'true', 'statusCode': 200, 'header': {}}

## Query single product

In [ ]:
%%time
sdk.querySingleProduct('0000002')

CPU times: user 6.71 ms, sys: 7.73 ms, total: 14.4 ms
Wall time: 83.5 ms


{'ib_prcode': '0000002',
 '1000': {'ib_cf_qty': 35,
  'new_ib_bs_stock_cv': 33,
  'lastUpdate': 1600567810.529301,
  'ib_prcode': '0000002',
  'ib_brcode': '1000'},
 '1001': {'ib_cf_qty': 32,
  'new_ib_bs_stock_cv': 30,
  'lastUpdate': 1600567810.529316,
  'ib_prcode': '0000002',
  'ib_brcode': '1001'},
 '1002': {'ib_cf_qty': 34,
  'new_ib_bs_stock_cv': 30,
  'lastUpdate': 1600567810.529318,
  'ib_prcode': '0000002',
  'ib_brcode': '1002'},
 'lastUpdate': 1600567810.529318}

## Query Branch

In [ ]:
%%time
result = sdk.queryBranch('1000')
#showing the first 2 result
list(iter(result.items()))[:2]

CPU times: user 360 ms, sys: 28.1 ms, total: 388 ms
Wall time: 600 ms


[('0000009',
  {'ib_cf_qty': 50,
   'new_ib_bs_stock_cv': 27,
   'lastUpdate': 1602338504.869655,
   'ib_prcode': '0000009',
   'ib_brcode': '1000'}),
 ('0000002',
  {'ib_cf_qty': 35,
   'new_ib_bs_stock_cv': 33,
   'lastUpdate': 1600567810.529301,
   'ib_prcode': '0000002',
   'ib_brcode': '1000'})]

## Query All

In [ ]:
%%time
result = sdk.queryAll()
list(iter(result.items()))[:2]

CPU times: user 2.34 s, sys: 90.4 ms, total: 2.43 s
Wall time: 2.77 s


[('0000009',
  {'ib_prcode': '0000009',
   '1000': {'ib_cf_qty': 50,
    'new_ib_bs_stock_cv': 27,
    'lastUpdate': 1602338504.869655,
    'ib_prcode': '0000009',
    'ib_brcode': '1000'},
   'lastUpdate': 1602338504.869655}),
 ('0000002',
  {'ib_prcode': '0000002',
   '1000': {'ib_cf_qty': 35,
    'new_ib_bs_stock_cv': 33,
    'lastUpdate': 1600567810.529301,
    'ib_prcode': '0000002',
    'ib_brcode': '1000'},
   '1001': {'ib_cf_qty': 32,
    'new_ib_bs_stock_cv': 30,
    'lastUpdate': 1600567810.529316,
    'ib_prcode': '0000002',
    'ib_brcode': '1001'},
   '1002': {'ib_cf_qty': 34,
    'new_ib_bs_stock_cv': 30,
    'lastUpdate': 1600567810.529318,
    'ib_prcode': '0000002',
    'ib_brcode': '1002'},
   'lastUpdate': 1600567810.529318})]